In [ ]:
import json
compFolder = '../DSS_2021/DataScienceSummit2021_Competition/dss_theater_datacomp_2021/'

In [ ]:
jsonFile = open(compFolder+'formattedTrainingData.json')

In [ ]:
trainingData = json.load(jsonFile)
jsonFile.close()

In [ ]:
from math import floor
from random import random,seed

In [ ]:
def countClass(classification, data):
    total = 0
    for elem in data:
        if data[elem]['TARGET_PANDEMIC_THTR_1'] == classification:
            total += 1
    return total     

In [ ]:
def countClasses(dataSet):
    classCounts = []
    classesCounted = []
    for data in dataSet:
        currItem = dataSet[data]
        if currItem['TARGET_PANDEMIC_THTR_1'] not in classesCounted:
            classesCounted.append(currItem['TARGET_PANDEMIC_THTR_1'])
            classCounts.append({currItem['TARGET_PANDEMIC_THTR_1']:{'count':countClass(currItem['TARGET_PANDEMIC_THTR_1'],dataSet)}})
    classCounts.append({'total':len(dataSet)})
    for i in range(0,len(classCounts)-1):
        for key in list(classCounts[i]):
            classCounts[i][key].update({'probability': classCounts[i][key]['count']/classCounts[len(classCounts)-1]['total']})
    return (classCounts, classesCounted)

In [ ]:
def getPossibleClasses(key,dataSet):
    possibleClasses = []
    for data in dataSet:
        if dataSet[data][key] not in possibleClasses:
            possibleClasses.append(dataSet[data][key])
    return possibleClasses

In [ ]:
def countAttributes(dataSet, categories):
    keyCounts = []
    initialItem = dataSet[list(dataSet.keys())[0]]
    keys = [k for k in initialItem]
    for key in initialItem:
        keyCounts.append({key:{}})
        for cat in categories:
            keyCounts[keys.index(key)][key].update({cat:{}})
            for classification in getPossibleClasses(key,dataSet):
                keyCounts[keys.index(key)][key][cat].update({classification:{'count':0}})
            keyCounts[keys.index(key)][key][cat].update({'total':0})
    for data in dataSet:
        currItem = dataSet[data]
        for key in currItem:
            keyCounts[keys.index(key)][key][currItem['TARGET_PANDEMIC_THTR_1']][currItem[key]]['count'] += 1
            keyCounts[keys.index(key)][key][currItem['TARGET_PANDEMIC_THTR_1']]['total'] += 1
    for i in range(0,len(keyCounts)):
        for key1 in keyCounts[i]:
            for key2 in keyCounts[i][key1]:
                for classification in getPossibleClasses(key1,dataSet):
                    if keyCounts[i][key1][key2]['total'] == 0:
                        keyCounts[i][key1][key2][classification].update({'probability':0})
                    else:
                        keyCounts[i][key1][key2][classification].update({'probability':keyCounts[i][key1][key2][classification]['count']/keyCounts[i][key1][key2]['total']})
    return keyCounts

In [ ]:
def naiveBayes(row,classCounts,attrCounts,possibleClasses):
    bayesProbabilities = []
    probs = []
    keys = [k for k in row]
    for classification in possibleClasses:
        bayesProb = 1
        for key in row:
            if key != 'TARGET_PANDEMIC_THTR_1':
                bayesProb *= attrCounts[keys.index(key)][key][classification][row[key]]['probability']
        bayesProbabilities.append({classification:bayesProb})
        probs.append(bayesProb)
#     print(bayesProbabilities)
    highestProbability = max(probs)
    return probs[1]

In [ ]:
testingSize = 1000
testing_data = {}
seed(0)
while(len(testing_data) < testingSize):
    ranNum = floor(len(trainingData)*random())
    testing_data[list(trainingData.keys())[ranNum]] = trainingData[list(trainingData.keys())[ranNum]]
classCounts = countClasses(trainingData)
print(classCounts)
attributeCounts = countAttributes(trainingData,classCounts[1])
# print(attributeCounts)
bayesClassifications = []
for row in testing_data:
    bayesClassifications.append(naiveBayes(testing_data[row], classCounts,attributeCounts,classCounts[1]))
print("Correct classifications: ", [testing_data[row]['TARGET_PANDEMIC_THTR_1'] for row in testing_data])
print("Model classifications: ", bayesClassifications)
numCorrect = 0
for i in range(0, len(testing_data)):
    if testing_data[list(testing_data.keys())[i]]['TARGET_PANDEMIC_THTR_1'] == bayesClassifications[i]:
        numCorrect += 1
print("Accuracy: ", (numCorrect/testingSize)*100, '%')